# **RAG based LLM application to translate legal documents from legal language to plain and simple English language.**

## Importing LangChain, Google BigQuery, Vertex AI for Gemini and Python libraries

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

from google.cloud import bigquery
from vertexai.preview.generative_models import GenerativeModel

import re

## Reading the Ontario Rental Act for Tenants, 2006 PDF version

In [10]:
loader = PyPDFLoader('ontario_rent_act.pdf')
docs = loader.load_and_split()

### Number of pages in the document

In [11]:
len(docs)

158

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 10,
    length_function = len,
    is_separator_regex = False,
)

In [13]:
texts = text_splitter.create_documents([each.page_content for each in docs])

### As expected number of chunks (585) is more than number of pages (158)

In [14]:
len(texts)

585

## Preparing data to make embeddings/vectors and store them in BigQuery table utilising BigQuery's new capability of Vector Search.

In [15]:
REGION = "US" 
DATASET = "vector_db"
TABLE = "on_vector_tbl"
PROJECT_ID = "image-genai"


### Using Vertex AI's text embedding model which creates vectors of about 768 dimensions. This is lesser than that of OpenAI's model but good enough for a POC.

In [16]:
embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

### Initialise BigQuery client and create dataset if it does not exist.

In [17]:
client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)

Dataset(DatasetReference('image-genai', 'vector_db'))

### We are using LangChain's BigQuery Vector search module.

In [18]:
store = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
    distance_strategy=DistanceStrategy.COSINE,
)

### Creating word embeddings and storing them in a BigQuery table.

In [19]:
data = [t.page_content for t in texts]
metadatas = [{"len": len(t.page_content)} for t in texts]

ids = store.add_texts(data, metadatas=metadatas)

print(len(ids))

585


### We use cosine similarity to get most similar vectors against a query, the query also is embedded using the same embedding model.

In [20]:
query = "Where is act being applied?"
query_vector = embedding.embed_query(query)
docs = store.similarity_search_by_vector(query_vector, k=3)
print(docs)

[Document(page_content='Provisions conflicting with Act void  \n4 (1)  Subject to subsection 12.1 (11) and section 194 , a provision in a tenancy agreement that is inconsistent with this Act or \nthe regul ations is void.  2006, c.  17, s.  4; 2017, c. 13, s. 1.  \nSame, Part V.1  \n(2)  Subject to section 194 , in any proceeding under Part V.1, a provision in an occupancy agreement in respect of a me mber \nunit or a provision in a by -law of a non -profit housing co -operative that is inconsistent with Part V.1, or with a provision in \nanother Part of this Act that applies to non -profit housing co -operatives and member units, does not apply, and the prov ision \nin this Act applies.  2013, c.  3, s. 23.  \nSection Amendments with date in force  (d/m/y)  \n2013, c. 3, s. 23  - 01/06/2014  \n2017, c. 13, s. 1  - 30/05/2017  \nExemptions from Act  \n5 This Act does not apply with respect to,  \n (a) living accommodation intended to be provided to the travelling or vacationing public 

## Preparing prompt for LLM

In [21]:
def augment_prompt(query: str):
    query_vector = embedding.embed_query(query)
    # get top 3 results from knowledge base
    results = store.similarity_search_by_vector(query_vector, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the question in detail.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

## We are using Google's LLM - Gemini

In [22]:
def generate(query):
    prompt = augment_prompt(query)
    model = GenerativeModel("gemini-1.0-pro-vision-001")
    responses = model.generate_content(
        prompt,
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.7,
            "top_p": 1
        },
        stream=False,
    )

    res = []
    print(responses.text)

    # for response in responses:
    #     print(response)
    #     # print(response.text, end="")

### Testing

In [23]:
generate("Where is act being applied?")

The Act applies to all tenancy agreements, occupancy agreements in respect of a member unit, and by-laws of a non-profit housing co-operative, except for living accommodation intended to be provided to the travelling or vacationing public or occupied for a seasonal or transient purpose.


In [28]:
generate("What is this act about?")

This act is about the regulation of residential tenancies in Ontario, Canada. It outlines the rights and responsibilities of landlords and tenants, including the selection of prospective tenants, the provision of information by landlords, the terms of tenancy agreements, and the resolution of disputes. The act also provides for exemptions from certain provisions for certain types of housing, such as social housing and non-profit housing co-operatives.


In [25]:
generate("Who seems to be benefitting from this act, landlord or the tenants?")

Based on the provided contexts, it seems that the Residential Tenancies Act, 2006 aims to protect the rights of tenants. The provisions mentioned in the contexts, such as compensation for increased rent and moving expenses, and the onus being on the landlord to prove that certain subsections apply, suggest that the act is designed to benefit tenants.


In [30]:
generate("What are some key rights which with I can protect myself as a tenant")

**Key Rights to Protect Yourself as a Tenant:**

* **Landlord's Responsibility to Repair:** Landlords are obligated to maintain the rental unit in a habitable condition, including repairs for issues such as plumbing, heating, and electrical systems. (Section 20)

* **Landlord's Responsibility Re Services:** Landlords must provide essential services such as water, heat, and electricity. (Section 21)

* **Landlord Not to Interfere with Reasonable Enjoyment:** Landlords cannot unreasonably interfere with your quiet enjoyment of the rental unit, including prohibiting you from having guests or pets without reasonable justification. (Section 22)

* **Landlord Not to Harass, Etc.:** Landlords are prohibited from engaging in harassment, obstruction, coercion, or intimidation against tenants. (Section 23)

* **Changing Locks:** Landlords cannot change locks on the rental unit without giving you notice and a key. (Section 24)

* **Privacy:** Landlords cannot enter the rental unit without your co

In [27]:
query = """Explain the below clause from my rental agreement

The tenant may assign or sublet the residential premises to another person with the consent of the landlord. A new
tenant must assume all of the rights and duties under the existing tenancy agreement, at the same rent. The landlord
must not charge a fee or receive a benefit, directly or indirectly, for giving this consent.
If a landlord unreasonably withholds consent to assign or sublet or charges a fee, the tenant may apply to the Ontario
Rent Tribunal for arbitration."""

In [31]:
generate(query)

The clause in your rental agreement is similar to the provisions outlined in the Residential Tenancies Act, 2006. Here's an explanation of the clause:

**Tenant's Right to Assign or Sublet:**
- The tenant has the right to assign (transfer) or sublet (rent out a portion of) the residential premises to another person.

**Landlord's Consent:**
- The tenant must obtain the landlord's consent before assigning or subletting the premises.

**Landlord's Obligations:**
- If the landlord consents to the assignment or sublet, the new tenant (assignee or subtenant) must assume all the rights and duties under the existing tenancy agreement.
- The landlord cannot charge a fee or receive any benefit, directly or indirectly, for giving this consent.

**Unreasonable Withholding of Consent:**
- The landlord cannot arbitrarily or unreasonably refuse consent to the assignment or sublet.
- If the landlord unreasonably withholds consent, the tenant can apply to the Ontario Rent Tribunal for arbitration.

**

In [32]:
query = '''Explain the clause below from my rental agreement

(a) Landlord’s Duties:
The landlord must provide and maintain the residential premises and residential property in a reasonable state of
decoration and repair, making the residential premises and the residential property suitable for occupation by a
reasonable tenant. The landlord must comply with health, safety, and housing standards required by law.
If the landlord is required to make a repair to comply with the above duties, the tenant may discuss it with the
landlord. If the landlord refuses to make the repair, the tenant may apply to the Ontario Rent Tribunal for the
completion and costs of the repair.
'''

In [33]:
generate(query)

**The clause you provided is similar to the landlord's responsibility to repair under section 20(1) of the Residential Tenancies Act, 2006 (RTA).**

**Here's an explanation of the clause and how it relates to the RTA:**

* **Landlord's Duties:** The landlord is responsible for providing and maintaining the residential premises and residential property in a reasonable state of decoration and repair. This means that the landlord must ensure that the rental unit is in a habitable condition and meets certain standards of cleanliness and repair.

* **Suitable for Occupation:** The landlord must make the residential premises and the residential property suitable for occupation by a reasonable tenant. This means that the rental unit must be in a condition that a reasonable person would find acceptable to live in.

* **Compliance with Standards:** The landlord must comply with health, safety, and housing standards required by law. This includes compliance with building codes, fire safety regul